In [2]:
import cv2
import numpy as np
import os
import sys
import time
import smtplib
import pywhatkit as kit
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

def send_email(subject, body, image_path):
    
    sender_email = os.environ.get('SENDER_MAIL')
    sender_password = os.environ.get('SENDER_PASSWORD')
    receiver_email = os.environ.get('RECEIVER_MAIL')

    # Creating message object instance
    message = MIMEMultipart()

    # Setting the message subject, body, and image attachment
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))
    
    try:
        with open(image_path, 'rb') as fp:
            img = MIMEImage(fp.read())
            img.add_header('Content-Disposition', 'attachment', filename='facelock.jpg')
            message.attach(img)
    except Exception as e:
        print("Error attaching image:", e)
        return

    # Create SMTP session
    mtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    mtpObj.starttls()
    mtpObj.login(sender_email, sender_password)

    # Sending email
    mtpObj.sendmail(sender_email, receiver_email, message.as_string())

    # Terminating the SMTP session
    mtpObj.quit()
    
    print("Email sent successfully!")



In [4]:
def send_whatsapp_message(message):
    kit.sendwhatmsg_instantly("+919834452882", message)
    print("WhatsApp message sent successfully!")


In [12]:
# Initializing LBPH face recognizer and loading the trained model

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')

# Initializing Haar cascade classifier for face detection

cascadePath = "haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# Initializing font for displaying text on image

font = cv2.FONT_HERSHEY_SIMPLEX

# Define IDs and names for recognized faces

names = ['Anonymous', 'Tushar', 'Dhoni', 'Akshay', 'Hulk', 'Virat','Adam']

# Initializing and starting real-time video capture

cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cam.set(3, 640) 
cam.set(4, 480) 

# Defining min window size to be recognized as a face

minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(int(minW), int(minH)))

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        # Check if confidence is less than 100 ==> "0" is perfect match 
        
        if confidence < 100:
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
            print("Access Granted with confidence score of", confidence)
            cam.release()
            cv2.destroyAllWindows()
            sys.exit()
        else:
            id = "Anonymous"
            confidence = "  {0}%".format(round(100 - confidence))
            #cropped = img[y:y+h, x:x+w]
            cropped = img[y-50:y+h+50, x-50:x+w+50]
            cv2.imwrite('unknown.jpg', cropped)
            print("Access Denied with confidence score of", confidence)
            send_email("Intruder Alert", "An Unknown face Detected!", "unknown.jpg")
            send_whatsapp_message("Intruder Alert! An Unknown face has been detected.")
            cam.release()
            cv2.destroyAllWindows()
            sys.exit()


    cv2.imshow('Face Detection',img) 

    k = cv2.waitKey(10) & 0xff # Press 'Enter Key' for exiting video
    if k == 13:
        break


print("\n Exiting Program...")
cam.release()
cv2.destroyAllWindows()


Access Denied with confidence score of   -36%
Email sent successfully!
WhatsApp message sent successfully!


SystemExit: 

In [8]:
os.environ['RECEIVER_MAIL'] = 'tushar.21910413@viit.ac.in'

In [9]:
os.environ['SENDER_MAIL'] = 'joshitushar210@gmail.com'

In [10]:
os.environ['SENDER_PASSWORD'] = 'oxqjaxzozycdebvy'

In [11]:
os.environ['CONTACT_NO'] = '+919834452882'

In [ ]:
# Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
            

In [ ]:
cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  

In [ ]:
if (confidence < 100):
            if id in range(len(names)):
                name = names[id]
            else:
                name = "Anonymous"
                confidence = "  {0}%".format(round(100 - confidence))
                
                # Crop the image and send email
                cropped = img[y:y+h, x:x+w]
                cv2.imwrite('unknown.jpg', cropped)
                send_email("Intruder Alert", "An Unknown face Detected!", "unknown.jpg")

                # Send WhatsApp message
                pw.sendwhatmsg(phone_num, message)  

        else:
            name = "Anonymous"
            confidence = "  {0}%".format(round(100 - confidence)